#load data as zipfile

#dimension reduction - PCA, UMAP, t-SNE 

#generating slightly different images with the same size

#model with https://stats.stackexchange.com/questions/148139/rules-for-selecting-convolutional-neural-network-hyperparameters and compare with https://keras.io/api/applications/resnet/